# O2c Iteradores, generadores, lambdas

## Comprensión de listas
Uno de los usos más comunes de un bucle `for` es almacenar valores en una lista:

In [1]:
cubos = []
for i in range(8):
    cubos.append(i**3)

print(cubos)

[0, 1, 8, 27, 64, 125, 216, 343]


Este mismo código se puede escribir de forma más elegante, en una sola línea, usando una comprensión de lista:

In [2]:
cubos = [i**3 for i in range(8)]

print(cubos)

[0, 1, 8, 27, 64, 125, 216, 343]


Se pueden filtrar elementos:

In [3]:
cubos_impares = [i**3 for i in range(8) if i%2 == 1]

print(cubos_impares)

[1, 27, 125, 343]


que sería equivalente al bucle

In [4]:
cubos_impares = []

for i in range(8):
    if i%2 == 1:
        cubos_impares.append(i**3)

print(cubos_impares)

[1, 27, 125, 343]


Además de comprensión de listas, también existen comprensión de conjuntos y de diccionarios, que funcionan de un modo similar:

In [6]:
letras = {letra for letra in 'palabra'}
print(letras)

{'p', 'b', 'l', 'a', 'r'}


In [7]:
cubos_impares = {i: i**3 for i in range(8) if i%2 == 1}
print(cubos_impares)

{1: 1, 3: 27, 5: 125, 7: 343}


## Iteradores

Un iterador es cualquier objeto que se pueda usar como `in` en un bucle `for`. Algunos tipos definidos por Python, como `list`, `tuple`, `set` y `str` son iteradores:

In [8]:
for letra in 'palabra':
    print(letra)

p
a
l
a
b
r
a


Otro iterador muy común es `range()`. En Python 3, `range()` no genera una lista de valores, sino que los genera uno a uno. Para generarlos todos de una vez, tenemos que convertir el `range` en una lista:

In [10]:
print(range(7))

print(list(range(7)))

range(0, 7)
[0, 1, 2, 3, 4, 5, 6]


En general, un iterador es cualquier objeto que tenga definido un método `__next__()`. Al iniciar cada paso del bucle, se llama a `__next__()`, y su valor devuelto se guarda en la variable del bucle. También debe tener un método `__iter__()` que devuelva un objeto iterable, que por lo general es `self`:

In [18]:
from time import sleep

class range_lento:
    def __init__(self):
        self.valor = 0

    def __next__(self):
        sleep(2.5)
        self.valor += 1
        return self.valor

    def __iter__(self):
        return self

In [17]:
for i in range_lento():
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


KeyboardInterrupt: 

¡Hemos creado un iterador infinito! Para señalar al bucle `for` cuando parar, hay que lanzar una excepción `StopIteration`:

In [52]:
class range_lento:
    def __init__(self, max):
        self.valor = 0
        self.max = max

    def __next__(self):
        if self.valor <= self.max:
            sleep(2.5)
            self.valor += 1
            return self.valor
        else:
            raise StopIteration

    def __iter__(self):
        return self

In [53]:
for i in range_lento(4):
    print(i)

1


In [50]:
class FloatRange:
    def __init__(self, start, stop, step=1.0):
        if start >= stop:
            raise ValueError("Invalid range")
        self.start = start
        self.stop = stop
        self.step = step

    def __iter__(self):
        n = self.start
        while n < self.stop:
            yield n
            n += self.step

    def __reversed__(self):
        n = self.stop - self.step
        while n >= self.start:
            yield n
            n -= self.step

In [51]:
for number in FloatRange(0.0, 5.0, 0.5):
    print(number)

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


## Generadores

Un generador es un tipo especial de fuunción que crea un iterador.

En una función ordinaria, una vez que se alcanza `return` (o `raise`), se sale de la función y python olvida completamnete el estado de sus variables internas, de modo que la siguiente vez que se llama a la función, se empieza desde el principio.

Un generador tiene una o más expresiones `yield`. Cuando se alcanza `yield`, se sale del generador, pero se conserva su estado interno. La próxima vez que se llama al generador, se empieza desde la línea siguiente al `yield`, conservando los valores de las variables internas.

In [23]:
def gen():
    x = 2+3
    yield 'Hola'
    yield x

for i in gen():
    print(i)

Hola
5


Un generador puede combinar varios `yield` y un `return`. Cuando se alcance el `return`, se acaba la ejecución del iterador:

In [24]:
def gen():
    x = 2+3
    yield 'Hola'
    yield x
    return 7
    yield 'a'

for i in gen():
    print(i)

Hola
5


Se pueden crear generadores con una sintaxis similar a la de una comprensión de lista, pero encerrados en paréntesis en vez de corchetes:

In [25]:
for i in (i**3 for i in range(8)):
    print(i)

0
1
8
27
64
125
216
343


Puede parecer similar a una comprensión de lista, pero la diferencia es que solamente se evalúa a cada paso del bucle. Compara estos dos códigos:

In [26]:
def cubo(x):
    sleep(2)
    return x**3

In [30]:
miscubos = (cubo(i) for i in range(8))
print("Generador creado")

for i in miscubos:
    print(i)

Generador creado
0
1
8
27
64
125
216
343


In [31]:
miscubos = [cubo(i) for i in range(8)]
print("Lista creada")


for i in miscubos:
    print(i)

Lista creada
0
1
8
27
64
125
216
343


## Manipulación de iteradores

El paquete `random` de la librería estándar contiene algunas funciones para trabajar con secuencias. Una secuencia es un objeto con una longitud determinada (implementa `__len__()`) y cuyos elementos se pueden obtener mediante índices numéricos (implementa `__getitem__()`). Las listas, tuplas y `range` son ejemplos de secuencias.

In [42]:
import random

# Elige un número aleatorio entre 15 y 20
random.choice(range(15, 20))

15

In [46]:
# Reordena los elementos
l = ['a', 'b', 'c', 'd']
random.shuffle(l)
print(l)

['d', 'c', 'a', 'b']


In [47]:
# Elige 4 elementos aleatoriamente (sin repetición)
random.sample(range(20), 4)

[13, 18, 8, 3]

In [48]:
# Elige 20 elementos aleatoriamente (con repetición)
random.choices(range(5), k=20)

[2, 2, 0, 4, 0, 3, 0, 4, 0, 3, 3, 0, 3, 1, 0, 3, 2, 3, 1, 3]

Python tiene una función para darle la vuelta a un iterador, `reversed`. Se aplica a las secuencias, o a los iteradores que implementen `__reveresed__()`:

In [54]:
for i in reversed(range(10)):
    print(i)

9
8
7
6
5
4
3
2
1
0


Vamos a crear nuestro propio iterador con `__reversed__()`:

In [66]:
class range_lento:
    def __init__(self, max):
        self.max = max
    
    def __iter__(self):
        i = 0
        while i < self.max:
            sleep(1.2)
            yield i
            i += 1

    def __reversed__(self):
        i = self.max - 1
        while i >= 0:
            sleep(1.2)
            yield i
            i -= 1

In [59]:
for i in reversed(range_lento(5)):
    print(i)

4
3
2
1
0


Otra función de python para manipular iteradores es `sorted`. Esta función produce una lista con todos los elementos producidos por el iterador, ordenados de menor a mayor. Si es necesario, calcula todos los elementos.

In [67]:
sorted(reversed(range_lento(5)))

[0, 1, 2, 3, 4]

Si no se pasa ningún argumento más, `sorted` compara los elementos del iterador entre sí (para valores numéricos por su valor, para texto según sus códigos ascii/unicode, etc). En concreto, significa que, cuando ordenamos palabras, las que empiecen por mayúscula aparecerán antes que las que empiezan por minúscula, independientemente del orden alfabético:

In [68]:
sorted(['ayer', 'Hoy', 'mañana'])

['Hoy', 'ayer', 'mañana']

Para evitarlo, hay que pasar a `sorted` un argumento `key` que se corresponda con la función usada para ordenar los elementos. En este caso, usaremos `str.lower`, que convierte un string a minúsculas:

In [69]:
sorted(['ayer', 'Hoy', 'mañana'], key=str.lower)

['ayer', 'Hoy', 'mañana']

Como otro ejemplo, vamos a ordenar las palabras por el número de veces que aparezca la letra 'a':

In [71]:
def numero_a(s):
    return len([letra for letra in s.lower() if letra == 'a'])

sorted(['ayer', 'Hoy', 'mañana'], key=numero_a)

['Hoy', 'ayer', 'mañana']